In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import statsmodels.api as sm
from scipy import stats
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
# # module imports
# from patsy import dmatrices
# import pandas as pd
# from sklearn.linear_model import LogisticRegression
# import statsmodels.discrete.discrete_model as sm

# # read in the data & create matrices
# df = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")
# y, X = dmatrices('admit ~ gre + gpa + C(rank)', df, return_type = 'dataframe')

# # sklearn output
# model = LogisticRegression(fit_intercept = False, C = 1e9)
# mdl = model.fit(X, y)
# model.coef_

# # sm
# logit = sm.Logit(y, X)
# logit.fit().params

In [2]:
df = pd.read_csv("../../data/ucla_sample.csv")
df.head()

,female,read,write,math,hon,femalexmath
0,0,57,52,41,0,0
1,1,68,59,53,0,53
2,0,44,33,54,0,0
3,0,63,44,47,0,0
4,0,47,52,57,0,0


In [9]:
df.groupby('female').agg({'hon': np.mean})

,hon
female,
0,0.186813
1,0.293578


In [23]:
model = LogisticRegression(solver='lbfgs', C=1e9).fit(df['female'].values.reshape(-1, 1), df['hon'])
df.shape, model.intercept_, model.coef_

((200, 6), array([-1.47085067]), array([[0.59278117]]))

In [20]:
np.exp(model.intercept_)

array([0.22972998])

In [14]:
import statsmodels.formula.api as sm

model = sm.Logit(df['hon'], df['female'].values.reshape(-1, 1))
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.645284
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                    hon   No. Observations:                  200
Model:                          Logit   Df Residuals:                      199
Method:                           MLE   Df Model:                            0
Date:                Sun, 03 Mar 2019   Pseudo R-squ.:                 -0.1590
Time:                        14:40:10   Log-Likelihood:                -129.06
converged:                       True   LL-Null:                       -111.36
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.8781      0.210     -4.175      0.000      -1.290      -0.466


In [2]:
# https://archive.ics.uci.edu/ml/datasets/Cervical+cancer+%28Risk+Factors%29
df = pd.read_csv("../../data/risk_factors_cervical_cancer.csv")
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0


In [41]:
df.columns

Index(['Age', 'Number of sexual partners', 'First sexual intercourse',
       'Num of pregnancies', 'Smokes', 'Smokes (years)', 'Smokes (packs/year)',
       'Hormonal Contraceptives', 'Hormonal Contraceptives (years)', 'IUD',
       'IUD (years)', 'STDs', 'STDs (number)', 'STDs:condylomatosis',
       'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
       'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis',
       'STDs:pelvic inflammatory disease', 'STDs:genital herpes',
       'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV',
       'STDs:Hepatitis B', 'STDs:HPV', 'STDs: Number of diagnosis',
       'STDs: Time since first diagnosis', 'STDs: Time since last diagnosis',
       'Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller',
       'Citology', 'Biopsy'],
      dtype='object')

In [79]:
df['Hormonal Contraceptives'].value_counts()

1.0    469
0.0    263
Name: Hormonal Contraceptives, dtype: int64

In [52]:
df.groupby('STDs: Number of diagnosis').agg({'Biopsy': np.mean})

,Biopsy
STDs: Number of diagnosis,
0,0.060741
1,0.166667
2,0.000000
3,0.000000


In [66]:
df['Biopsy'].mean()

0.07103825136612021

In [60]:
# df['STDs (number)'].value_counts()
df = df.loc[df['Smokes'] != '?']
df = df.loc[df['STDs (number)'] != '?']
df = df.loc[df['Hormonal Contraceptives'] != '?']

In [80]:
def _numeric_transformer(X):
    return pd.DataFrame([X]).T

col_encoders = {
    'Hormonal Contraceptives': Pipeline(
                # StandardScaler needs a 2d np.array
                [('wrapper', FunctionTransformer(_numeric_transformer, validate=False, accept_sparse=True)),]
                #('scaler', OneHotEncoder(categories='auto'))]
    ),
#     'STDs (number)': Pipeline(
#                 # StandardScaler needs a 2d np.array
#                 [('wrapper', FunctionTransformer(_numeric_transformer, validate=False, accept_sparse=True)),
#                  ('scaler', OneHotEncoder(categories='auto'))]
#     )
}

columns = ['Hormonal Contraceptives']

transformers = ColumnTransformer([(col, col_encoders[col], col) for col in columns])

estimator = LogisticRegression(solver='lbfgs')

model = Pipeline([('transformers', transformers), ('estimator', estimator)])
model

Pipeline(memory=None,
     steps=[('transformers', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('Hormonal Contraceptives', Pipeline(memory=None,
     steps=[('wrapper', FunctionTransformer(accept_sparse=True, check_inverse=True,
          fun...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [81]:
model.fit(df[columns], df['Biopsy'])

Pipeline(memory=None,
     steps=[('transformers', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('Hormonal Contraceptives', Pipeline(memory=None,
     steps=[('wrapper', FunctionTransformer(accept_sparse=True, check_inverse=True,
          fun...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [82]:
model.named_steps['estimator'].intercept_[0], model.named_steps['estimator'].coef_

(-2.607508693319554, array([[0.05672268]]))

In [88]:
np.exp(model.named_steps['estimator'].intercept_[0]), df.loc[df['Hormonal Contraceptives'] == '1.0', 'Biopsy'].mean(), df.loc[df['Hormonal Contraceptives'] == '0.0', 'Biopsy'].mean()

(0.07371796925288078, 0.07249466950959488, 0.06844106463878327)

In [83]:
np.exp(model.named_steps['estimator'].intercept_[0])

0.07371796925288078

In [76]:
# https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/
# P(y=1)/(1-P(y=1)) for no Hormonal Contraceptives
np.exp(model.named_steps['estimator'].intercept_[0] + model.named_steps['estimator'].coef_[0][0])

0.0735990917568669